<a href="https://colab.research.google.com/github/7201krap/PYTORCH_project/blob/main/selectivity_4_HL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch.utils.data import DataLoader, Subset
from torchvision import transforms
import numpy as np
from google.colab import drive

import os
import subprocess as sp
from torchvision.datasets.mnist import MNIST, read_image_file, read_label_file
from torchvision.datasets.utils import extract_archive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.manual_seed(1234)
np.random.seed(1234)

In [ ]:
def patched_download(self):
    """wget patched download method.
    """
    if self._check_exists():
        return

    os.makedirs(self.raw_folder, exist_ok=True)
    os.makedirs(self.processed_folder, exist_ok=True)

    # download files
    for url, md5 in self.resources:
        filename = url.rpartition('/')[2]
        download_root = os.path.expanduser(self.raw_folder)
        extract_root = None
        remove_finished = False

        if extract_root is None:
            extract_root = download_root
        if not filename:
            filename = os.path.basename(url)
        
        # Use wget to download archives
        sp.run(["wget", url, "-P", download_root])

        archive = os.path.join(download_root, filename)
        print("Extracting {} to {}".format(archive, extract_root))
        extract_archive(archive, extract_root, remove_finished)

    # process and save as torch files
    print('Processing...')

    training_set = (
        read_image_file(os.path.join(self.raw_folder, 'train-images-idx3-ubyte')),
        read_label_file(os.path.join(self.raw_folder, 'train-labels-idx1-ubyte'))
    )
    test_set = (
        read_image_file(os.path.join(self.raw_folder, 't10k-images-idx3-ubyte')),
        read_label_file(os.path.join(self.raw_folder, 't10k-labels-idx1-ubyte'))
    )
    with open(os.path.join(self.processed_folder, self.training_file), 'wb') as f:
        torch.save(training_set, f)
    with open(os.path.join(self.processed_folder, self.test_file), 'wb') as f:
        torch.save(test_set, f)

    print('Done!')


MNIST.download = patched_download

In [ ]:
mnist_trainset = MNIST(root='./data', train=True, 
                                download=True, 
                                transform=transforms.Compose([transforms.ToTensor()]))

mnist_testset  = MNIST(root='./data', 
                                train=False, 
                                download=True, 
                                transform=transforms.Compose([transforms.ToTensor()]))

train_dataloader = torch.utils.data.DataLoader(mnist_trainset, 
                                               batch_size=50, 
                                               shuffle=True)

test_dataloader  = torch.utils.data.DataLoader(mnist_testset, 
                                               batch_size=50, 
                                               shuffle=False)

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw
Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw
Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw
Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
# ************* modify this section for later use *************
# Define the model 
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()

        # modify this section for later use 
        self.linear_1 = torch.nn.Linear(784, 256)
        self.linear_2 = torch.nn.Linear(256, 256)
        self.linear_3 = torch.nn.Linear(256, 256)
        self.linear_4 = torch.nn.Linear(256, 256)
        self.linear_5 = torch.nn.Linear(256, 10)
        self.sigmoid12  = torch.nn.Sigmoid()
        self.sigmoid23  = torch.nn.Sigmoid()
        self.sigmoid34  = torch.nn.Sigmoid()
        self.sigmoid45  = torch.nn.Sigmoid()

        self.layer_activations = dict()

    def forward(self, x):

        # modify this section for later use 
        x = x.reshape(x.size(0), -1)
        x = self.linear_1(x)
        x = self.sigmoid12(x)
        x = self.linear_2(x)
        x = self.sigmoid23(x)
        x = self.linear_3(x)
        x = self.sigmoid34(x)
        x = self.linear_4(x)
        x = self.sigmoid45(x)
        pred = self.linear_5(x)
        return pred
# ************* modify this section for later use *************

In [ ]:
def get_activation(model, layer_name):    
    def hook(module, input, output):
        model.layer_activations[layer_name] = output
    return hook

In [ ]:
def selectivity(hidden_layer_each_neuron):
    __selectivity__ = list()
    # I will now try to find the average of each class for each neuron.
    # check out the next cell 
    avg_activations = [dict() for x in range(256)]
    for i, neuron in enumerate(hidden_layer_each_neuron):
        for k, v in neuron.items():
            # v is the list of activations for hidden layer's neuron k 
            avg_activations[i][k] = sum(v) / float(len(v))

    # generate 256 lists to get only values in avg_activations
    only_activation_vals = [list() for x in range(256)]

    # get only values from avg_activations
    for i, avg_activation in enumerate(avg_activations):
        for value in avg_activation.values():
            only_activation_vals[i].append(value)

    for activation_val in only_activation_vals:
        # find u_max 
        u_max = np.max(activation_val)

        # find u_minus_max 
        u_minus_max = (np.sum(activation_val) - u_max) / 9

        # find selectivity 
        selectivity = (u_max - u_minus_max) / (u_max + u_minus_max)

        # append selectivity value to selectivity
        __selectivity__.append(selectivity)

    avg_selectivity = np.average(__selectivity__)
    std_selectivity = np.std(__selectivity__)
                                 
    return avg_selectivity, std_selectivity

In [ ]:
def model_factory(optimizer_name):
    '''
    optimizer_name : choose one of Adagrad, Adadelta, SGD, and Adam 

    '''
    my_model = Model()
    print("my_model:", my_model)
    my_model.to(device)

    # ************* modify this section for later use *************
    # chagen sigmoid34 an 's34'
    my_model.sigmoid12.register_forward_hook(get_activation(my_model, 's12'))
    my_model.sigmoid23.register_forward_hook(get_activation(my_model, 's23'))
    my_model.sigmoid34.register_forward_hook(get_activation(my_model, 's34'))
    my_model.sigmoid45.register_forward_hook(get_activation(my_model, 's45'))
    # ************* modify this section for later use *************

    if optimizer_name == 'Adadelta':
        my_optimizer = torch.optim.Adadelta(my_model.parameters(), lr=1.0)

    elif optimizer_name == 'Adagrad':
        my_optimizer = torch.optim.Adagrad(my_model.parameters(), lr=0.1)

    elif optimizer_name == 'SGD':
        my_optimizer = torch.optim.SGD(my_model.parameters(), lr=0.1)

    elif optimizer_name == 'Adam':
        my_optimizer = torch.optim.Adam(my_model.parameters(), lr=0.001)

    else:
        print("ERROR")
    
    print("my_optimizer:", my_optimizer)
    test_acc, selectivity_list_avg, selectivity_list_std, selec_12_avg, selec_12_std, selec_23_avg, selec_23_std, selec_34_avg, selec_34_std, selec_45_avg, selec_45_std = selectivity_trainer(optimizer=my_optimizer, model=my_model)
    # ************* modify this section for later use *************
    # change name of the file 
    file_saver = open(f"4HL_selectivity_new_{optimizer_name}.txt", "w")
    # ************* modify this section for later use *************
    file_saver.write(str(test_acc)+'\n'+str(selectivity_list_avg)+'\n'+str(selectivity_list_std)+'\n'+str(selec_12_avg)+'\n'+str(selec_12_std)+'\n'+str(selec_23_avg)+'\n'+str(selec_23_std)+'\n'+str(selec_34_avg)+'\n'+str(selec_34_std)+'\n'+str(selec_45_avg)+'\n'+str(selec_45_std)+'\n\n')
    file_saver.close()

    # ************* modify this section for later use *************
    if optimizer_name == 'Adadelta':
        !cp 4HL_selectivity_new_Adadelta.txt /content/drive/MyDrive
    
    elif optimizer_name == 'Adagrad':
        !cp 4HL_selectivity_new_Adagrad.txt /content/drive/MyDrive

    elif optimizer_name == 'SGD':
        !cp 4HL_selectivity_new_SGD.txt /content/drive/MyDrive

    elif optimizer_name == 'Adam':
        !cp 4HL_selectivity_new_Adam.txt /content/drive/MyDrive
    # ************* modify this section for later use *************

    else:
        print("ERROR")

In [ ]:
# ************* modify this section for later use *************
# add a parameter to the function and calculate avg and std. Do not forget to change division by 2, 3, 4, or 5 
def avg_std_calculator(_hidden_layer_each_neuron_12, _hidden_layer_each_neuron_23, _hidden_layer_each_neuron_34, _hidden_layer_each_neuron_45):

    avg_selectivity12, std_selectivity12 = selectivity(_hidden_layer_each_neuron_12)
    avg_selectivity23, std_selectivity23 = selectivity(_hidden_layer_each_neuron_23)
    avg_selectivity34, std_selectivity34 = selectivity(_hidden_layer_each_neuron_34)
    avg_selectivity45, std_selectivity45 = selectivity(_hidden_layer_each_neuron_45)

    final_selectivity_avg = (avg_selectivity12 + avg_selectivity23 + avg_selectivity34 + avg_selectivity45) / 4
    final_selecvitity_std = (std_selectivity12 + std_selectivity23 + std_selectivity34 + std_selectivity45) / 4

    return final_selectivity_avg, final_selecvitity_std, avg_selectivity12, std_selectivity12, avg_selectivity23, std_selectivity23, avg_selectivity34, std_selectivity34, avg_selectivity45, std_selectivity45 
# ************* modify this section for later use *************

In [ ]:
no_epochs = 50
def selectivity_trainer(optimizer, model):

    criterion = torch.nn.CrossEntropyLoss()
    train_loss = list()
    test_loss  = list()
    test_acc   = list()

    best_test_loss = 1

    selectivity_avg_list = list()
    selectivity_std_list = list()

    selec_12_avg = list()
    selec_12_std = list()
    selec_23_avg = list()
    selec_23_std = list()
    selec_34_avg = list()
    selec_34_std = list()
    selec_45_avg = list()
    selec_45_std = list()

    for epoch in range(no_epochs):

        # ************* modify this section for later use *************
        hidden_layer_each_neuron_12 = [{0:[], 1:[], 2:[], 3:[], 4:[], 5:[], 6:[], 7:[], 8:[], 9:[]} for x in range(256)]
        hidden_layer_each_neuron_12 = np.array(hidden_layer_each_neuron_12)

        hidden_layer_each_neuron_23 = [{0:[], 1:[], 2:[], 3:[], 4:[], 5:[], 6:[], 7:[], 8:[], 9:[]} for x in range(256)]
        hidden_layer_each_neuron_23 = np.array(hidden_layer_each_neuron_23)

        hidden_layer_each_neuron_34 = [{0:[], 1:[], 2:[], 3:[], 4:[], 5:[], 6:[], 7:[], 8:[], 9:[]} for x in range(256)]
        hidden_layer_each_neuron_34 = np.array(hidden_layer_each_neuron_34)

        hidden_layer_each_neuron_45 = [{0:[], 1:[], 2:[], 3:[], 4:[], 5:[], 6:[], 7:[], 8:[], 9:[]} for x in range(256)]
        hidden_layer_each_neuron_45 = np.array(hidden_layer_each_neuron_45)
        # ************* modify this section for later use *************


        total_train_loss = 0
        total_test_loss = 0

        # training
        # set up training mode 
        model.train()

        for itr, (images, labels) in enumerate(train_dataloader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            pred = model(images)

            loss = criterion(pred, labels)
            total_train_loss += loss.item()

            loss.backward()
            optimizer.step()

        total_train_loss = total_train_loss / (itr + 1)
        train_loss.append(total_train_loss)

        # testing 
        # change to evaluation mode 
        model.eval()
        total = 0
        for itr, (images, labels) in enumerate(test_dataloader):
            images, labels = images.to(device), labels.to(device)

            pred = model(images)

            loss = criterion(pred, labels)
            total_test_loss += loss.item()

            # we now need softmax because we are testing.
            pred = torch.nn.functional.softmax(pred, dim=1)
            for i, p in enumerate(pred):
                if labels[i] == torch.max(p.data, 0)[1]:
                    total = total + 1

            # ************* modify this section for later use *************
            # Do not forget to change hidden_layer_each_neuron_12 name 
            for activation, label in zip(model.layer_activations['s12'], labels):
                label = label.item()
                with torch.no_grad():
                    activation = activation.numpy()
                for i in range(256):    
                    hidden_layer_each_neuron_12[i][label].append(activation[i])

            for activation, label in zip(model.layer_activations['s23'], labels):
                label = label.item()
                with torch.no_grad():
                    activation = activation.numpy()
                for i in range(256):    
                    hidden_layer_each_neuron_23[i][label].append(activation[i])
            
            for activation, label in zip(model.layer_activations['s34'], labels):
                label = label.item()
                with torch.no_grad():
                    activation = activation.numpy()
                for i in range(256):    
                    hidden_layer_each_neuron_34[i][label].append(activation[i])

            for activation, label in zip(model.layer_activations['s45'], labels):
                label = label.item()
                with torch.no_grad():
                    activation = activation.numpy()
                for i in range(256):    
                    hidden_layer_each_neuron_45[i][label].append(activation[i])

        # Add one more parameter 
        selectivity_avg, selectivity_std, avg_selectivity12, std_selectivity12, avg_selectivity23, std_selectivity23, avg_selectivity34, std_selectivity34, avg_selectivity45, std_selectivity45 = avg_std_calculator(hidden_layer_each_neuron_12, hidden_layer_each_neuron_23, hidden_layer_each_neuron_34, hidden_layer_each_neuron_45)
        # ************* modify this section for later use *************
        
        selec_12_avg.append(avg_selectivity12)
        selec_12_std.append(std_selectivity12)
        selec_23_avg.append(avg_selectivity23)
        selec_23_std.append(std_selectivity23)
        selec_34_avg.append(avg_selectivity34)
        selec_34_std.append(std_selectivity34)
        selec_45_avg.append(avg_selectivity45)
        selec_45_std.append(std_selectivity45)

        selectivity_avg_list.append(selectivity_avg)
        selectivity_std_list.append(selectivity_std)

        # caculate accuracy 
        accuracy = total / len(mnist_testset)

        # append accuracy here
        test_acc.append(accuracy)

        # append test loss here 
        total_test_loss = total_test_loss / (itr + 1)
        test_loss.append(total_test_loss)

        print('\nEpoch: {}/{}, Train Loss: {:.8f}, Test Loss: {:.8f}, Test Accuracy: {:.8f}'.format(epoch + 1, no_epochs, total_train_loss, total_test_loss, accuracy))


    return test_acc, selectivity_avg_list, selectivity_std_list, selec_12_avg, selec_12_std, selec_23_avg, selec_23_std, selec_34_avg, selec_34_std, selec_45_avg, selec_45_std

# AdaGrad

In [ ]:
model_factory('Adagrad')

my_model: Model(
  (linear_1): Linear(in_features=784, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=256, bias=True)
  (linear_3): Linear(in_features=256, out_features=256, bias=True)
  (linear_4): Linear(in_features=256, out_features=256, bias=True)
  (linear_5): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid12): Sigmoid()
  (sigmoid23): Sigmoid()
  (sigmoid34): Sigmoid()
  (sigmoid45): Sigmoid()
)
my_optimizer: Adagrad (
Parameter Group 0
    eps: 1e-10
    initial_accumulator_value: 0
    lr: 0.1
    lr_decay: 0
    weight_decay: 0
)

Epoch: 1/50, Train Loss: 2.40975734, Test Loss: 2.31905772, Test Accuracy: 0.09580000

Epoch: 2/50, Train Loss: 2.32717139, Test Loss: 2.32578379, Test Accuracy: 0.11350000

Epoch: 3/50, Train Loss: 2.32179257, Test Loss: 2.31983709, Test Accuracy: 0.09820000

Epoch: 4/50, Train Loss: 2.31935933, Test Loss: 2.30957953, Test Accuracy: 0.10280000

Epoch: 5/50, Train Loss: 2.31706264, Test Loss: 2.31821671